In [1]:
%matplotlib qt

import neuroseries as nts

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import scipy.ndimage
import os

import scipy.signal
import scipy.stats

import bk.load as bk
import bk.plot
import bk.compute

def TTLtoTimes(TTL,Fs = 20000):
    
    if isinstance(TTL[0],(np.bool_,bool)):
        TTL = list(map(int,TTL))
    
    diff_TTL = np.diff(TTL)
    
    t_start = np.where(diff_TTL == 1)[0]
    t_end = np.where(diff_TTL == -1)[0]
    t_TTL = np.array([np.mean(interval) for interval in zip(t_start,t_end)])
    
    return t_TTL/Fs

def TTL_to_intervals(TTL,Fs = 20000):
    if isinstance(TTL[0],(np.bool_,bool)):
        TTL = list(map(int,TTL))
    
    diff_TTL = np.diff(TTL)
    
    t_start = np.where(diff_TTL == 1)[0]
    t_end = np.where(diff_TTL == -1)[0]
    t_TTL = np.array([np.mean(interval) for interval in zip(t_start,t_end)])
    
    return (t_start/Fs,t_end/Fs)

def speed(pos):
    
    x_speed = np.diff(pos['x'])/np.diff(pos.index)
    y_speed = np.diff(pos['y'])/np.diff(pos.index)
    
    v = np.sqrt(x_speed**2 + y_speed**2)
    
    return v

def load_pos():
    import pandas as pd
    files = os.listdir()
    for f in files: 
        if f.endswith('.h5'): 
            filename = f
    
    data = pd.read_hdf(filename)
    data = data[data.keys()[0][0]]
    return data

def load_digitalin(channels,Fs = 20000):
    import pandas as pd
    
    digital_word = np.fromfile('digitalin.dat','uint16')
    sample = len(digital_word)
    time = np.arange(0,sample)
    time = time/Fs

    chan = {}
    
    for i in range(len(channels)):
        chan.update({channels[i]:(digital_word & 2**i)>0})

    return time,chan

# def load_digitalin(path,nchannels=16,Fs = 20000):
#     import pandas as pd
    
#     digital_word = np.fromfile(path,'uint16')
#     sample = len(digital_word)
#     time = np.arange(0,sample)
#     time = time/Fs

    
#     for i in range(nchannels):
#         if i == 0: data = (digital_word & 2**i)>0
#         else: data = np.vstack((data,(digital_word & 2**i)>0))

#     return data



In [2]:
def lfp(start, stop, n_channels=90, channel=64, frequency=1250.0, precision='int16',verbose = False):
    
    p = session+".lfp"
    if verbose:
        print('Load LFP from ' + p)
    # From Guillaume viejo
    import neuroseries as nts
    bytes_size = 2
    start_index = int(start*frequency*n_channels*bytes_size)
    stop_index = int(stop*frequency*n_channels*bytes_size)
    #In order not to read after the file
    if stop_index > os.path.getsize(p): stop_index = os.path.getsize(p)
    fp = np.memmap(p, np.int16, 'r', start_index, shape = (stop_index - start_index)//bytes_size)
    data = np.array(fp).reshape(len(fp)//n_channels, n_channels)

    if type(channel) is not list:
        timestep = np.arange(0, len(data))/frequency+start
        return nts.Tsd(timestep, data[:,channel], time_units = 's')
    elif type(channel) is list:
        timestep = np.arange(0, len(data))/frequency+start
        return nts.TsdFrame(timestep, data[:,channel], time_units = 's')

In [27]:
session = 'Rat03-20201002'

In [5]:
os.chdir('R:/Rat25/Rat25-210402/Rat25_210402_113108/')

In [7]:
d = bk.load.digitalin('digitalin.dat')

In [10]:
plt.plot(d[0,0:2000000:16])

In [1]:
plt.plot(t[::16],digital['Exp'][::16])

NameError: name 'plt' is not defined

In [2]:
os.chdir('D:\Rat03\Rat03-20201002')

In [29]:
# data = bk.load.digitalin('digitalin.dat')
l = lfp(0,500,102,5)

In [18]:
t = np.arange(0,len(data[0,:])/20000,1/20000,dtype = 'float16')

In [35]:
plt.plot(l.as_units('s'))
plt.plot(t[0:20000*10],data[0,0:20000*10]*1000)
plt.xlim(0,10)

(0.0, 10.0)

In [20]:
t.shape

(155964960,)

In [15]:
import numpy as np

In [24]:
plt.plot(t[::2],data[0,::2])
plt.axvline(107352/1000,color = 'red')
plt.axvline(7370748/1000,color = 'red')
plt.axvline(7582031.9999999/1000,color = 'red')

In [ ]:
def DLC_pos(filtered = True):
    """
    Load position from DLC files (*.h5) and returns it as a nts.TsdFrame
    """
    
    files = os.listdir()
    for f in files:
        if filtered and f.endswith('filtered.h5'): 
            filename = f
            break
        if not filtered and not f.endswith('filtered.h5') and f.endswith('.h5'):
            filename = f
            break
            
    data = pd.read_hdf(filename)
    data = data[data.keys()[0][0]]
    
    TTL = bk.load.digitalin('digitalin.dat')[0,:]
    tf = np.array(bk.compute.TTL_to_times(TTL))
    
    if len(tf)>len(data):
        tf = np.delete(tf,-1)
    
    data.index = tf * 1_000_000
    pos = nts.TsdFrame(data)
    return pos
        

In [6]:
path = 'Y:\classical_fear_conditionning\Rat01-20200630'
subpath = os.listdir(path)

os.chdir(os.path.join(path,subpath[0]))
pos = bk.load.DLC_pos()
speed = bk.compute.speed(pos,2,columns_to_drop=['e_tail'])


In [9]:
plt.figure()
fr = bk.compute.freezing_intervals(speed,18,mode='multiple_speed',clean = True)
bk.plot.intervals(fr,'orange')

In [2]:
start = [0,5,10,12]
end = [4.5,6,11,12.1]

a = nts.IntervalSet(start,end,time_units='s')

In [5]:
a.drop_short_intervals(4.6,time_units='s')

,start,end


In [8]:
plt.show()

In [27]:
pos = nts.TsdFrame(np.linspace(0,50,len(data)),data)

In [8]:
# path = 'Y:/ExampleSession/rat01_200630_114732'
# path = 'Y:/ExampleSession/rat01_200630_171900'
# path = 'Y:/ExampleSession/rat01_200630_183149/'

# path = 'Y:/ExampleSession/RAT02/rat02_200707_114611'


# i = 1
# title = ['Fear','Tone','Context']

# os.chdir(path)
# step = 1

# time,data = load_digitalin(['TTL','Shock','Tone'])
# pos = load_pos()

# s = speed(pos['snout'])
# s = scipy.ndimage.gaussian_filter1d(s,1)

# tf = TTLtoTimes(data['TTL'])
# time = time-tf[0]
# tf = np.delete(tf,-1)

# pos.index = tf-tf[0]
# time = time[0:-1:step]

# fig = plt.subplot(1,1,i)
# plt.tight_layout()
# plt.title(title[i-1])
# plt.plot(time,data['TTL'][0:-1:step])
# plt.plot(time,data['Shock'][0:-1:step] * 3)
# plt.plot(time,data['Tone'][0:-1:step]*2)
# plt.plot(pos.index[:-1],s/15)
# plt.legend(['TTL','Shock','Tone','Speed'])
# plt.xlabel('Time (s)')
# #plt.vlines(x=tf,ymin = 0, ymax = 3,color='red')
# i += 1

# plt.show()

In [9]:
def freezing(speed,treshold, mode='single_speed'):
    """
        BK 8/11/20
        Input 
            speed: speed vector as output by bk.compute.speed (not yet implemented. But it's an nts.frame)
            treshold: arbritary units
    """
    
    if mode.lower() =='single_speed':
        fs =  1/scipy.stats.mode(np.diff(speed.index)).mode[0]
        freezing = speed.values<threshold
        if freezing[0] == 1: freezing[0] = 0
        if freezing[-1] == 1: freezing = np.append(freezing,0)

        dfreeze = np.diff(freezing.astype(np.int8))
        start = np.where(dfreeze == 1)[0]/fs + speed.index[0]
        end = np.where(dfreeze == -1)[0]/fs + speed.index[0]
    elif mode.lower() == 'multiple_speed':
        fs =  1/scipy.stats.mode(np.diff(speed.index)).mode[0]
        freezing = np.array((np.sum(all_speed.as_units('s'),axis = 1))/5 < threshold)
        if freezing[0] == 1: freezing[0] = 0
        if freezing[-1] == 1: freezing = np.append(freezing,0)

        dfreeze = np.diff(freezing.astype(np.int8))
        start = np.where(dfreeze == 1)[0]/fs + speed.index[0]
        end = np.where(dfreeze == -1)[0]/fs + speed.index[0]
        #print('not implemented')
    elif mode.lower() == 'pca':
        print('not implanted')
    else:
        print('Mode not recognized')
        return False
    freezing_intervals = nts.IntervalSet(start,end)
    return freezing_intervals

def consolidate_intervals(freezing_intervals,t_min):
    starts = []
    ends = []
    for i,z in enumerate(zip(freezing_intervals.as_units('s').start, freezing_intervals.as_units('s').end)):
        s=z[0]
        e=z[1]
        if ends:
            if s<ends[-1]: continue
        if i == freezing_intervals.shape[0]-1:
            starts.append(s)
            ends.append(e)
        elif freezing_intervals.as_units('s').start[i+1]-e<t_min:
            starts.append(s)
            j=i
            while freezing_intervals.as_units('s').start[j+1]-freezing_intervals.as_units('s').end[j]<t_min and j<(len(freezing_intervals.as_units('s').start)-2):
                j=j+1
            if j<(len(freezing_intervals.as_units('s').start)-2):
                ends.append(freezing_intervals.as_units('s').end[j])
            else:
                ends.append(freezing_intervals.as_units('s').end[j+1])
        else:
            starts.append(s)
            ends.append(e)
        
    freeze = nts.IntervalSet(starts,ends,time_units = 's')
    return freeze

def freezing_video(video_path,output_file,tf,freezing_intervals):
    
    """
        video_path : path to the video to be displaying
        outputfile : path to the video to written
        tf : vector of time containing timing of each frame
        freezing intervals : Intervals when the animal is freezing (as nts.Interval_Set)
    """
    import cv2

    if os.path.exists(output_file):
        print(output_file,'already exist, please delete manually')
        return
    
    tf = nts.Ts(tf-tf[0],time_units='s')
    freezing_frames = np.where(freezing_intervals.in_interval(tf)>=0)[0]
    fs =  1/scipy.stats.mode(np.diff(tf.as_units('s').index)).mode[0]
    cap  = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    
    nf = 0
    out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), fs, (frame_width,frame_height))
    while True:
        
        ret,frame = cap.read()
        if ret == True:
            if nf in freezing_frames: frame = cv2.circle(frame,(25,25),10,(0,0,255),20)

            cv2.imshow(video_path,frame)
            out.write(frame)
            if cv2.waitKey(1) & 0xFF == ord('q'): 
                break
            nf +=1
        else:
            break
    cap.release()
    out.release()
    
    return True

In [10]:
# path = '/media/billel/DATA/ExampleSession/RAT01/'
path = 'Y:/ExampleSession/RAT01/'
subpath = os.listdir(path)

os.chdir(os.path.join(path,subpath[0]))
# os.chdir('Y:/ExampleSession/RAT01/rat01_200630_171900')
step = 20

time,data = load_digitalin(['TTL','Shock','Tone'])

tf = TTLtoTimes(data['TTL'])
tf = np.delete(tf,-1)

time = time-tf[0]

pos = load_pos()
pos.index = tf-tf[0]


s = speed(pos['snout'])
s = scipy.ndimage.gaussian_filter1d(s,2)
print(s)
s = nts.Tsd(pos.index.values[:-1],s,time_units='s')


time = time[0:-1:step]
fig = plt.subplot(1,1,1)
plt.tight_layout()
plt.title('Freezing')
plt.plot(time,data['TTL'][0:-1:step])
plt.plot(time,data['Shock'][0:-1:step] * 3)
plt.plot(time,data['Tone'][0:-1:step]*2)
plt.plot(pos.index[:-1],s.values/1500)
plt.legend(['TTL','Shock','Tone','Speed'])
plt.xlabel('Time (s)')
#plt.vlines(x=tf,ymin = 0, ymax = 3,color='red')

freezing_intervals = bk.compute.freezing_intervals(s,50)
freezing_intervals = consolidate_intervals(freezing_intervals,0.300)
freezing_intervals = freezing_intervals[(freezing_intervals.as_units('s').end-freezing_intervals.as_units('s').start)>2.5]

bk.plot.intervals(freezing_intervals,col='orange')

plt.show()

# freezing_video(video_path='rat01-fear.mp4',output_file='freezing.avi',tf = tf,freezing_intervals = freezing_intervals)

[11.6357088  12.24101139 12.79317226 ...  8.76313895  9.21187629
  9.39674661]


In [13]:
a = bk.compute.tone_intervals(data['Tone'])

In [11]:
freezing_intervals

,start,end
0,0,11163875
1,11630425,18295425
24,66416725,69782550
31,79880025,85012075
36,92176950,96775800
49,116270925,120603175
53,123769050,128634500
73,158293750,173223350
74,175222850,184020650
76,186953250,191885350


In [7]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [31]:
# plt.plot(data['Tone'])

Tone = TTL_to_intervals(data['Tone'])

In [32]:
Tone = nts.IntervalSet(Tone[0],Tone[1],time_units='s')

In [33]:
Tone = Tone.merge_close_intervals(1,time_units='s')
Tone = Tone.drop_short_intervals(1,time_units = 's')

In [34]:
bk.plot.intervals(Tone,col = 'Orange')

In [39]:
body = []
for i in pos:
    body.append(i[0])
body = np.unique(body)
print(body)

['base_tail' 'end_tail' 'left_ear' 'right_ear' 'snout']


In [26]:
all_speed = np.empty((len(pos)-1,5))
i = 0
for b in body:
    all_speed[:,i] = speed(pos[b])
    i +=1
plt.figure()
all_speed = all_speed[1500:,:]
all_speed = scipy.ndimage.gaussian_filter1d(all_speed,2,axis=0)
plt.plot(all_speed)
plt.legend(body)
all_speed = nts.TsdFrame(t = pos.index.values[1500:-1],d = all_speed, time_units = 's',columns = body)

In [221]:
y = np.sin(np.arange(0,6,0.01))
np_conv = np.convolve(y,gaussian,'same')/10
scipy_conv = scipy.signal.fftconvolve(y,gaussian,'same')
scipy_filter = scipy.ndimage.gaussian_filter1d(y,10)

In [222]:
plt.plot(y)
plt.plot(np_conv)
plt.plot(scipy_conv)
plt.plot(scipy_filter)

plt.legend(['ori','np','scipy conv','scipy ndimage'])

In [134]:
speedos = nts.TsdFrame(t = tf)

In [144]:
speedos

,base_tail,end_tail,left_ear,right_ear,snout
Time (us),,,,,
0,37.583033,724.806739,1190.980166,683.069592,1358.278619
33325,37.569126,734.036610,1183.248852,657.804164,1361.083096
66650,37.516557,749.482695,1167.085870,610.186259,1366.088526
99975,37.380474,765.833503,1141.208035,545.493193,1372.104397
133325,37.102946,776.710469,1104.110209,470.389676,1377.368749
...,...,...,...,...,...
622420199,8.080557,27.901076,15.482479,13.704811,12.590928
622453524,8.057923,28.757154,15.204255,13.513517,12.524467
622486850,8.046167,29.480066,14.942245,13.340893,12.481720


In [13]:
import sklearn.manifold as skm
import scipy.stats
from matplotlib import cm

In [40]:
tsne = skm.TSNE(n_components=2).fit_transform(all_speed)

In [42]:
c = scipy.stats.zscore(all_speed.snout.values)
# c = np.linspace(0,1,len(all_speed))
viridis = cm.get_cmap('viridis')
plt.clf()
plt.scatter(tsne[:,0],tsne[:,1],cmap=plt.get_cmap('viridis'),c =viridis(c) )
plt.colorbar()

In [1]:
def plot_frame_video(video_path):
    
    """
        video_path : path to the video to be displaying
        outputfile : path to the video to written
        tf : vector of time containing timing of each frame
        freezing intervals : Intervals when the animal is freezing (as nts.Interval_Set)
    """

    

    import cv2
    cap  = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    while True:
        ret,frame = cap.read()
        if ret == True:
            cv2.imshow(video_path,frame)
        else:
            break
            
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
        
    cap.release()
    
    return True

In [3]:
plot_frame_video('/media/sf_Y/classical_fear_conditionning/Rat09-20201118/Rat09_201118_110003/Basler_acA1300-200uc__23039139__20201118_110005549.mp4')

True

In [103]:
import sklearn.manifold as skm

In [86]:
from matplotlib import cm

In [ ]:
def GetStartExp(Exp,Fs = 20000):
    
    if isinstance(Exp[0],(np.bool_,bool)):
        Exp = list(map(int,Exp))
    
    diff_Exp = np.diff(Exp)
    
    t_start = np.where(diff_Exp == 1)[0]    
    return t_start/Fs

In [ ]:
files = os.listdir()
for f in files:
    if f.endswith('.mp4'):
        nFrame = CountFrames(f)

        if len(tf)>nFrame:
            tf = np.delete(tf,-1)
            break